In [ ]:
"""Thực hành phân loại tin tức (CNN, RNN, LSTM).ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1r9wfT4z6yO2epyiAVx1C_Aw3IYtEAqvb

# Phân loại văn bản báo tiếng việt

## Import các thư viện cần dùng
"""

In [ ]:
import json
import requests
from pprint import pprint
import re
import string
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import warnings
import sys
import torch

In [ ]:
# Kiểm tra GPU có khả dụng không
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cpu':
    raise RuntimeError("No GPU found")

In [ ]:
print(f'Using device: {device}')

In [ ]:
# Đặt random seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
"""## Tiền sử lý dữ liệu

### Tải file json chứa dữ liệu
"""

In [ ]:
# URL trực tiếp đến file JSON thô trên GitHub
url = 'https://raw.githubusercontent.com/letangphuquy/AI4E/master/3-ml-crawler/data/news_final.json'

In [ ]:
# Tải nội dung của file
response = requests.get(url)

In [ ]:
# Kiểm tra mã trạng thái phản hồi để đảm bảo rằng yêu cầu thành công
if response.status_code == 200:
    try:
        data = response.json()  # Đọc nội dung file JSON

        # Hiển thị phần tử đầu tiên của file JSON
        if isinstance(data, list) and len(data) > 0:
            pprint(json.dumps(data[0], indent=4, ensure_ascii=False))
        else:
            print("Dữ liệu JSON không phải là một danh sách hoặc danh sách trống.")

    except json.JSONDecodeError as e:
        print("JSONDecodeError: Không thể phân tích nội dung JSON.")
        print("Nội dung phản hồi không phải là JSON hợp lệ:")
        print(response.text)  # In ra nội dung phản hồi để kiểm tra
else:
    print(f"Yêu cầu thất bại với mã trạng thái {response.status_code}")
    print("Nội dung phản hồi:", response.text)

In [ ]:
"""### Xem các đầu mục"""

In [ ]:
# Trích xuất các danh mục (categories)
categories = {}
for article in data:
    cat = article['metadata']['cat']
    if not cat in categories:
        categories[cat] = {'cnt': 0, 'subcat': set()}
    categories[cat]['cnt'] += 1
    categories[cat]['subcat'].add(article['metadata']['subcat'])

In [ ]:
for category, info in categories.items():
    print(f"Category: {category}, Count: {info}")

In [ ]:
"""### Chuyển đổi đầu mục"""

In [ ]:
# Từ điển ánh xạ các categories hiện tại về 12 loại
# Một số category khá "broad" do hiện tượng mất cân bằng dữ liệu
categories_include = {
    "Thời sự": ["Chính trị", "Tuần Việt Nam"],
    "Pháp luật": [],
    "Thế giới": [],
    "Kinh tế": ["Kinh doanh", "Bất động sản", "Thị trường tiêu dùng", "Việc làm"],
    "Xã hội": ["Văn hóa", "Dân tộc - tôn giáo", "Nông thôn mới", "Dân tộc thiểu số và miền núi", "Nội dung chuyên đề", "An sinh"],
    "Giáo dục": [],
    "Sức khỏe": [], # Sức khỏe - Y tế
    "Đời sống": ["Bảo vệ người tiêu dùng", "Nhân ái", "Tình yêu"],
    "KHCN": ["Thông tin và truyền thông", "Công nghiệp hỗ trợ", "Khoa học", "Sức mạnh số", "Số hóa", "Khoa học - công nghệ"],
    "Thể thao": [],
    "Giải trí": ["Du lịch"]
}

In [ ]:
# Lọc và chuyển đổi categories
filtered_data = []
for article in data:
    old_category = article['metadata']['cat']
    if old_category not in categories_include:
        new_category = ""
        for category, subcategories in categories_include.items():
            flag = False
            for subcat in subcategories:
                if subcat in old_category: flag = True; new_category = category
            if flag: break
        if not new_category: continue
        article['metadata']['cat'] = new_category
        filtered_data.append(article)
    else: filtered_data.append(article)

In [ ]:
# In ra các bài báo sau khi lọc và chuyển đổi categories
print(f"Filtered articles count: {len(filtered_data)}")
# Trích xuất các danh mục (categories)
categories = [article['metadata']['cat'] for article in filtered_data]

In [ ]:
# Đếm số lượng từng danh mục
category_counts = {}
for category in categories:
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

In [ ]:
# Vẽ biểu đồ cột
plt.figure(figsize=(10, 6))  # Điều chỉnh kích thước biểu đồ nhỏ hơn
ax = sns.barplot(x=list(category_counts.keys()), y=list(category_counts.values()))
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Article Count per Category')
plt.xticks(rotation=45)

In [ ]:
# Thêm số lượng ở trên mỗi cột
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
plt.show()
plt.savefig('category-distribution.png')

In [ ]:
# Tạo một defaultdict để lưu các văn bản theo từng category
category_data = defaultdict(list)
for article in filtered_data:
    category_data[article['metadata']['cat']].append(article)

In [ ]:
# Xác định số lượng mẫu tối đa cho mỗi category
number_sample = 12000
max_samples_per_category = number_sample // len(category_data)

In [ ]:
# Chọn ngẫu nhiên các mẫu từ mỗi category, giới hạn bởi max_samples_per_category
balanced_data = []
for category, items in category_data.items():
    if len(items) > max_samples_per_category:
        balanced_data.extend(random.sample(items, max_samples_per_category))
    else:
        balanced_data.extend(items)

In [ ]:
# Nếu số lượng mẫu không đủ number_sample, bổ sung thêm mẫu ngẫu nhiên từ balanced_data
if len(balanced_data) < number_sample:
    additional_samples = random.sample(balanced_data, number_sample - len(balanced_data))
    balanced_data.extend(additional_samples)

In [ ]:
filtered_data_to_use = balanced_data[:number_sample if number_sample < len(balanced_data) else len(balanced_data)]
print(f"Filtered articles to use count: {len(filtered_data_to_use)}")

In [ ]:
# Đếm số lượng từng danh mục
category_counts = defaultdict(int)
for item in filtered_data_to_use:
    category_counts[item['metadata']['cat']] += 1

In [ ]:
# Vẽ biểu đồ cột
plt.figure(figsize=(10, 6))  # Điều chỉnh kích thước biểu đồ nhỏ hơn
ax = sns.barplot(x=list(category_counts.keys()), y=list(category_counts.values()))
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Article Count per Category')
plt.xticks(rotation=45)

In [ ]:
# Thêm số lượng ở trên mỗi cột
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
plt.show()
plt.savefig('balanced-category-distribution.png')

In [ ]:
"""### Làm sạch hơn dữ liệu"""

In [ ]:
# Danh sách stop words tiếng Việt
stop_words = set([
    "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện",
    "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều",
    "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "này",
    "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra", "rằng",
    "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước", "từ", "từng",
    "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
])

In [ ]:
# Trích xuất dữ liệu từ các bài báo đã được lọc
texts = []
labels = []

In [ ]:
# Lặp qua từng bài báo trong dữ liệu đã lọc
for article in filtered_data_to_use:
    # Kết hợp tiêu đề và nội dung bài báo
    text = article.get('title', '') + ' ' + article.get('description', '') + ' ' + article.get('content', '')
    # Lấy nhãn (category) của bài báo
    label = article['metadata']['cat']
    # Thêm văn bản và nhãn vào danh sách tương ứng
    texts.append(text)
    labels.append(label)

In [ ]:
# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Chuyển văn bản thành chữ thường
    text = text.lower()
    # Loại bỏ dấu câu
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    # Tách văn bản thành các từ (tokens)
    tokens = text.split()
    # Đếm tổng số từ ban đầu
    total_tokens = len(tokens)
    # Loại bỏ các từ thuộc danh sách stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Tính phần trăm các từ stop words đã bị loại bỏ
    removed_stop_words_percent = (total_tokens - len(tokens)) / total_tokens * 100
    # Trả về văn bản đã được xử lý và phần trăm từ stop words đã bị loại bỏ
    return ' '.join(tokens), removed_stop_words_percent

In [ ]:
# Danh sách chứa các văn bản đã qua tiền xử lý
processed_texts = []
# Danh sách chứa phần trăm các từ stop words đã bị loại bỏ trong từng văn bản
sum_stop_words_percents = 0

In [ ]:
# Sử dụng tqdm để hiển thị thanh tiến trình khi xử lý các văn bản
tqdm_iter = tqdm(texts, desc="Processing texts")

In [ ]:
# Lặp qua từng văn bản trong danh sách văn bản gốc
for i, text in enumerate(tqdm_iter):
    # Tiền xử lý văn bản và tính phần trăm từ stop words đã bị loại bỏ
    processed_text, stop_words_percent = preprocess_text(text)
    # Thêm văn bản đã qua tiền xử lý vào danh sách
    processed_texts.append(processed_text)
    # Thêm phần trăm từ stop words đã bị loại bỏ vào danh sách
    sum_stop_words_percents += stop_words_percent
    # Tính phần trăm trung bình của các từ stop words đã bị loại bỏ
    avg_stop_words_percent = sum_stop_words_percents / (i+1)
    # Cập nhật thanh tiến trình với phần trăm trung bình hiện tại
    tqdm_iter.set_postfix({"avg_stop_words_percent": f"{avg_stop_words_percent:.2f}%"})

In [ ]:
# Cập nhật danh sách văn bản gốc với các văn bản đã qua tiền xử lý
texts = processed_texts
# Tính phần trăm trung bình cuối cùng của các từ stop words đã bị loại bỏ
average_stop_words_percent = sum_stop_words_percents / len(texts)

In [ ]:
# In ra phần trăm trung bình của các từ stop words đã bị loại bỏ
print(f"Average percentage of stop words removed: {average_stop_words_percent:.2f}%")

In [ ]:
print("Văn bản sau sử lý: ", texts[0])
print("Nhãn của văn bản: ", labels[0])

In [ ]:
"""## Huấn luyện và đánh giá các mô hình truyền thống

### Tạo dữ liệu huấn luyện và kiểm thử cho các mô hình truyền thống
"""

In [ ]:
# Khai báo mảng label y
y = np.array(labels)

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
texts_train, texts_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [ ]:
print("Nhãn gốc: ", y_train[:5])
print("Nhãn sau khi mã hóa: ", y_train_enc[:5])

In [ ]:
# Shuffle the training data
texts_train, y_train_enc = shuffle(texts_train, y_train_enc, random_state=42)

In [ ]:
# Vectorize texts using TF-IDF for SVM
tfidf_vectorizer = TfidfVectorizer(max_features=5000, tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_train_tfidf = tfidf_vectorizer.fit_transform(texts_train)
X_test_tfidf = tfidf_vectorizer.transform(texts_test)

In [ ]:
print("Kích thức của tập train: ", X_train_tfidf.shape)

In [ ]:
"""### Khởi tạo, huấn luyện và đánh giá các mô hình truyền thống

Mô hình Naive Bayes
"""

In [ ]:
# Naive Bayes model
nb_model = MultinomialNB(alpha=1.0)

In [ ]:
# Fit dữ liệu
nb_model.fit(X_train_tfidf, y_train_enc)

In [ ]:
# Dự đoán
y_pred_enc_nb = nb_model.predict(X_test_tfidf)
y_pred_nb = le.inverse_transform(y_pred_enc_nb)

In [ ]:
# Báo cáo phân loại cho Naive Bayes
report_nb = classification_report(y_test, y_pred_nb, output_dict=True, zero_division=0)

In [ ]:
"""Mô hình Logistic Regression"""

In [ ]:
### YOUR START CODE HERE ###
# Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)

In [ ]:
# Fit dữ liệu
lr_model.fit(X_train_tfidf, y_train_enc)

In [ ]:
# Dự đoán
y_pred_enc_lr = lr_model.predict(X_test_tfidf)
y_pred_lr = le.inverse_transform(y_pred_enc_lr)

In [ ]:
# Báo cáo phân loại cho Logistic Regression
report_lr = classification_report(y_test, y_pred_lr, output_dict=True, zero_division=0)
### YOUR END CODE HERE ###

In [ ]:
"""Mô hình Random Forest"""

In [ ]:
### YOUR START CODE HERE ###
# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Fit dữ liệu
rf_model.fit(X_train_tfidf, y_train_enc)
# Dự đoán
y_pred_enc_rf = rf_model.predict(X_test_tfidf)
y_pred_rf = le.inverse_transform(y_pred_enc_rf)

In [ ]:
# Báo cáo phân loại cho Logistic Regression
report_rf = classification_report(y_test, y_pred_rf, output_dict=True, zero_division=0)
### YOUR END CODE HERE ###

In [ ]:
"""Mô hình SVM"""

In [ ]:
### YOUR START CODE HERE ###
# SVM model
svc_model = SVC(kernel='linear')

In [ ]:
# Fit dữ liệu
svc_model.fit(X_train_tfidf, y_train_enc)
# Dự đoán
y_pred_enc_svc = svc_model.predict(X_test_tfidf)
y_pred_svc = le.inverse_transform(y_pred_enc_svc)

In [ ]:
# Báo cáo phân loại cho Logistic Regression
report_svc = classification_report(y_test, y_pred_svc, output_dict=True, zero_division=0)
### YOUR END CODE HERE ###

In [ ]:
"""### Vẽ biểu đồ"""

In [ ]:
# Tạo danh sách các báo cáo phân loại
reports = {
    'Naive Bayes': report_nb,
    'Logistic Regression': report_lr,
    'Random Forest': report_rf,
    'SVC': report_svc
}

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 16))
fig_cm, axs_cm = plt.subplots(2, 2, figsize=(16, 16))

In [ ]:
for i, (name, report) in enumerate(reports.items()):
    categories = list(report.keys())[:-3]  # exclude 'accuracy', 'macro avg', 'weighted avg'
    precisions = [report[cat]['precision'] for cat in categories]
    recalls = [report[cat]['recall'] for cat in categories]
    f1_scores = [report[cat]['f1-score'] for cat in categories]

    # In ra các thông số
    print(f'\n{name} Metrics:')
    for category in categories:
        print(f"{category}: Precision={report[category]['precision']:.2f}, Recall={report[category]['recall']:.2f}, F1-Score={report[category]['f1-score']:.2f}")

    # In ra độ chính xác trung bình
    accuracy = report['accuracy']
    print(f'Overall Accuracy: {accuracy:.2f}')

    # Vẽ biểu đồ đường
    axs[i // 2, i % 2].plot(categories, precisions, marker='o', label='Precision', color='blue')
    axs[i // 2, i % 2].plot(categories, recalls, marker='o', label='Recall', color='green')
    axs[i // 2, i % 2].plot(categories, f1_scores, marker='o', label='F1-Score', color='red')
    axs[i // 2, i % 2].set_ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1
    axs[i // 2, i % 2].set_title(f'{name} Metrics')
    axs[i // 2, i % 2].set_ylabel('Score')
    axs[i // 2, i % 2].legend()
    axs[i // 2, i % 2].tick_params(axis='x', rotation=45)

    # Vẽ ma trận nhầm lẫn
    if name == 'Naive Bayes':
        cm = confusion_matrix(y_test, y_pred_nb, labels=le.classes_)
    elif name == 'Logistic Regression':
        cm = confusion_matrix(y_test, y_pred_lr, labels=le.classes_)
    elif name == 'Random Forest':
        cm = confusion_matrix(y_test, y_pred_rf, labels=le.classes_)
    elif name == 'SVC':
        cm = confusion_matrix(y_test, y_pred_svc, labels=le.classes_)

    sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_, cmap='Blues', ax=axs_cm[i // 2, i % 2])
    axs_cm[i // 2, i % 2].set_title(f'{name} Confusion Matrix')
    axs_cm[i // 2, i % 2].set_xlabel('Predicted')
    axs_cm[i // 2, i % 2].set_ylabel('True')

In [ ]:
plt.tight_layout()
plt.show()
plt.savefig('traditional-ml-models.png')

In [ ]:
# In ra nhãn dự đoán và nhãn thực tế
print("Nội dung bài báo: ", texts_train[0])
print("Nhãn dự đoán:" , le.inverse_transform(rf_model.predict(X_test_tfidf[:1])))
print("Nhãn thực tế: ", le.inverse_transform(y_train_enc[:1]))

In [ ]:
"""## Huấn luyện và đánh giá các mô hình học sâu

### Tạo dữ liệu huấn luyện và kiểm thử cho các mô hình học sâu

Tải mô hình phobert dùng để chuyển word thành vector
"""

In [ ]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

In [ ]:
# Tải PhoBERT Tokenizer và Model
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
embeder = AutoModel.from_pretrained('vinai/phobert-base')

In [ ]:
# Chuyển mô hình sang GPU nếu có
embeder.to(device)

In [ ]:
"""Hàm chuyển dãy word thành dãy vector"""

In [ ]:
# Chuyển đổi từ thành vector.
def encode_texts(texts, tokenizer, embeder, max_length=256):
    encoded_texts = []  # Danh sách để lưu văn bản sau khi mã hóa.

    for text in tqdm(texts, desc="Chuyển từ thành vector"):
        # Chuyển văn bản thành các tensor đầu vào cho mô hình, cắt hoặc padding đến độ dài tối đa.
        inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length").to(device)

        # Không tính gradient vì chỉ thực hiện suy luận (inference), không huấn luyện.
        with torch.no_grad():
            outputs = embeder(**inputs)  # Truyền các tensor đầu vào qua mô hình để lấy các vector mã hóa.

        # Lấy vector mã hóa từ `last_hidden_state` và loại bỏ chiều batch.
        encoded_texts.append(outputs.last_hidden_state.squeeze(0))

    # Chuyển danh sách các tensor thành một tensor duy nhất.
    return torch.stack(encoded_texts)

In [ ]:
test = ["Xin chào", "Đây là chương trình AI4E"]
encode_texts(test, tokenizer, embeder).shape

In [ ]:
"""Chuyển đổi dữ liệu"""

In [ ]:
# Vectorize texts using phobert
X_train_phobert = encode_texts(texts_train, tokenizer, embeder)
X_test_phobert = encode_texts(texts_test, tokenizer, embeder)

In [ ]:
"""Tạo Dataloader"""

In [ ]:
# Khai báo lớp dùng để tạo dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [ ]:
# Tạo Dataset cho tập huấn luyện và tập kiểm thử
train_dataset = TextDataset(X_train_phobert, y_train_enc)
test_dataset = TextDataset(X_test_phobert, y_test_enc)

In [ ]:
# Tạo DataLoader cho tập huấn luyện và tập kiểm thử
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""### Định nghĩa các lớp mô hình học sâu"""

In [ ]:
# Xây dựng mô hình RNN
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNClassifier, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.num_layers = num_layers

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Xây dựng mô hình LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.num_layers = num_layers

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Xây dựng mô hình CNN
class CNNClassifier(nn.Module):
    def __init__(self, input_size, output_size, num_filters, kernel_size, max_length=256):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        self.pool = nn.MaxPool1d(kernel_size=max_length - kernel_size + 1)
        self.fc = nn.Linear(num_filters, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Đổi thứ tự để phù hợp với Conv1d (batch_size, input_size, max_length)
        x = self.conv1(x)
        x = self.pool(x).squeeze(2)
        x = self.fc(x)
        return x

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, input_size, output_size, num_filters, kernel_size, max_length=256):
        super(CNNClassifier, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size)
        self.bn1 = nn.BatchNorm1d(num_filters)
        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters*2, kernel_size=kernel_size)
        # self.bn2 = nn.BatchNorm1d(num_filters*2)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.drop = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(num_filters*2 * ((max_length - 2*(kernel_size-1)) // 2), 256)
        self.fc2 = nn.Linear(256, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Adjust input dimensions for Conv1d
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.drop(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
"""### Khai báo các mô hình học sâu"""

In [ ]:
# Thiết lập các thông số
input_size = 768  # Kích thước của vector ẩn PhoBERT
output_size = len(le.classes_) # Số lượng lớp

In [ ]:
### YOUR START CODE HERE ###
hidden_size = 512
num_filters = 10
kernel_size = 5
### YOUR END CODE HERE ###

In [ ]:
# Khởi tạo các mô hình
deep_models = {
    'CNN': CNNClassifier(input_size, output_size, num_filters, kernel_size, max_length=256).to(device),
    'RNN': RNNClassifier(input_size, hidden_size, output_size, num_layers=3).to(device),
    'LSTM': LSTMClassifier(input_size, hidden_size, output_size, num_layers=1).to(device),
}

In [ ]:
"""### Huấn luyện và đánh giá các mô hình học sâu"""

In [ ]:
# Hàm huấn luyện và đánh giá mô hình
def train_and_evaluate(model, train_loader, test_loader, num_epochs=10, criterion=None, optimizer_fn=None, scheduler_fn=None):
    # Khởi tạo optimizer
    optimizer = optimizer_fn(model.parameters())

    # Khởi tạo scheduler
    scheduler = scheduler_fn(optimizer)

    for epoch in range(num_epochs):
        model.train()
        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if scheduler is not None:
            scheduler.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

In [ ]:
"""Lựa chọn criterion, optimizer, và scheduler"""

In [ ]:
### YOUR START CODE HERE ###
# Khai báo criterion
criterion = nn.CrossEntropyLoss()

In [ ]:
# Hàm tạo optimizer
def optimizer_fn(params):
   return torch.optim.Adam(params, lr=0.001)

In [ ]:
# Hàm tạo scheduler
def scheduler_fn(optimizer):
    return torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
num_epochs = 20
### YOUR START CODE HERE ###

In [ ]:
"""### Vẽ biểu đồ"""

In [ ]:
# Hình vẽ kết quả của mô hình sau khi train
fig, axs = plt.subplots(2, 2, figsize=(16, 16))
fig_cm, axs_cm = plt.subplots(2, 2, figsize=(16, 16))

In [ ]:
# Lưu trữ F1 scores của các mô hình
model_f1_scores = {}

In [ ]:
for i, (name, model) in enumerate(deep_models.items()):
    print(f"Training {name} model...")
    y_test, y_pred = train_and_evaluate(model, train_loader, test_loader, num_epochs,
                                        criterion=criterion, optimizer_fn=optimizer_fn, scheduler_fn=scheduler_fn)
    y_test = le.inverse_transform(y_test)
    y_pred = le.inverse_transform(y_pred)

    # Tạo báo cáo phân loại
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    categories = list(report.keys())[:-3]  # exclude 'accuracy', 'macro avg', 'weighted avg'

    # Tạo dữ liệu để vẽ biểu đồ
    precisions = [report[cat]['precision'] for cat in categories]
    recalls = [report[cat]['recall'] for cat in categories]
    f1_scores = [report[cat]['f1-score'] for cat in categories]
    model_f1_scores[name] = f1_scores

    # In ra các thông số
    print(f'\n{name} Metrics:')
    for category in categories:
        print(f"{category}: Precision={report[category]['precision']:.2f}, Recall={report[category]['recall']:.2f}, F1-Score={report[category]['f1-score']:.2f}")

    # In ra độ chính xác trung bình
    accuracy = report['accuracy']
    print(f'Overall Accuracy: {accuracy:.2f}\n')

    # Vẽ biểu đồ đường
    axs[i // 2, i % 2].plot(categories, precisions, marker='o', label='Precision', color='blue')
    axs[i // 2, i % 2].plot(categories, recalls, marker='o', label='Recall', color='green')
    axs[i // 2, i % 2].plot(categories, f1_scores, marker='o', label='F1-Score', color='red')
    axs[i // 2, i % 2].set_ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1
    axs[i // 2, i % 2].set_title(f'{name} Metrics')
    axs[i // 2, i % 2].set_ylabel('Score')
    axs[i // 2, i % 2].legend()
    axs[i // 2, i % 2].tick_params(axis='x', rotation=45)

    # Vẽ ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred, labels=le.classes_)
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_, cmap='Blues', ax=axs_cm[i // 2, i % 2])
    axs_cm[i // 2, i % 2].set_title(f'{name} Confusion Matrix')
    axs_cm[i // 2, i % 2].set_xlabel('Predicted')
    axs_cm[i // 2, i % 2].set_ylabel('True')

In [ ]:
# Vẽ F1 scores của cả 3 mô hình lên subplot thứ 4
for name, f1_scores in model_f1_scores.items():
    axs[1, 1].plot(categories, f1_scores, marker='o', label=f'{name} F1-Score')
axs[1, 1].set_ylim(0, 1)
axs[1, 1].set_title('F1 Scores Comparison')
axs[1, 1].set_ylabel('F1 Score')
axs[1, 1].legend()
axs[1, 1].tick_params(axis='x', rotation=45)

In [ ]:
plt.tight_layout()
plt.show()
plt.savefig('modern-dl-models.png')

In [ ]:
# In nhãn dự đoán và nhãn thực tế
print("Văn bản đầu vào: ", texts_test[0])
print("Nhãn dự đoán: ", y_pred[0])
print("Nhãn thực tế: ", y_test[0])

In [ ]:
"""## Bài tập về nhà
Gán nhãn lại mục metadata.cat trong file text_no_cat.json, tải file đã gán nhãn lên trên github và nộp lại link.

Link file text_no_cat: https://github.com/nmquang003/AI4E/blob/main/test_news_no_cat.json

Link file categories: https://github.com/nmquang003/AI4E/blob/main/categories.txt

### Tải Test dataset
"""

In [ ]:
# URL trực tiếp đến file JSON thô trên GitHub
url = 'https://github.com/nmquang003/AI4E/blob/main/test_news_no_cat.json?raw=true'

In [ ]:
# Tải nội dung của file
response = requests.get(url)

In [ ]:
# Kiểm tra mã trạng thái phản hồi để đảm bảo rằng yêu cầu thành công
if response.status_code == 200:
    try:
        data = response.json()  # Đọc nội dung file JSON
        # Hiển thị phần tử đầu tiên của file JSON
        if isinstance(data, list) and len(data) > 0:
            pprint(json.dumps(data[0], indent=4, ensure_ascii=False))
        else:
            print("Dữ liệu JSON không phải là một danh sách hoặc danh sách trống.")

    except json.JSONDecodeError as e:
        print("JSONDecodeError: Không thể phân tích nội dung JSON.")
        print("Nội dung phản hồi không phải là JSON hợp lệ:")
        print(response.text)  # In ra nội dung phản hồi để kiểm tra
else:
    print(f"Yêu cầu thất bại với mã trạng thái {response.status_code}")
    print("Nội dung phản hồi:", response.text)

In [ ]:
"""### Làm sạch dữ liệu"""

In [ ]:
texts = []
for article in data:
    text = article.get('title', '') + ' ' + article.get('description', '') + ' ' + article.get('content', '')
    texts.append(text)

In [ ]:
# Hàm tiền xử lý văn bản
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    total_tokens = len(tokens)
    tokens = [token for token in tokens if token not in stop_words]
    removed_stop_words_percent = (total_tokens - len(tokens)) / total_tokens * 100
    return ' '.join(tokens), removed_stop_words_percent

In [ ]:
processed_texts = []
sum_stop_words_percents = 0

In [ ]:
# Sử dụng tqdm để hiển thị thanh tiến trình khi xử lý các văn bản
tqdm_iter = tqdm(texts, desc="Processing texts")

In [ ]:
for i, text in enumerate(tqdm_iter):
    processed_text, stop_words_percent = preprocess_text(text)
    processed_texts.append(processed_text)
    sum_stop_words_percents += stop_words_percent
    avg_stop_words_percent = sum_stop_words_percents / (i+1)
    tqdm_iter.set_postfix({"avg_stop_words_percent": f"{avg_stop_words_percent:.2f}%"})

In [ ]:
# Cập nhật danh sách văn bản gốc với các văn bản đã qua tiền xử lý
texts = processed_texts
average_stop_words_percent = sum_stop_words_percents / len(texts)

In [ ]:
# In ra phần trăm trung bình của các từ stop words đã bị loại bỏ
print(f"Average percentage of stop words removed: {average_stop_words_percent:.2f}%")

In [ ]:
"""### Tiền xử lý PhoBERT"""

In [ ]:
X_test_phobert = encode_texts(texts_test, tokenizer, embeder)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

In [ ]:
test_dataset = TestDataset(X_test_phobert)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""### Gán nhãn dữ liệu bằng mô hình"""

In [ ]:
model = deep_models['LSTM']
model.eval()

In [ ]:
all_preds = []
with torch.no_grad():
    for texts in test_loader:
        texts = texts.to(device)
        outputs = model(texts)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
for i, pred in enumerate(all_preds):
    data[i]['metadata']['cat'] = le.inverse_transform([pred])[0]

In [ ]:
"""### Lưu kết quả"""

In [ ]:
with open('labeled_data.json', 'w', encoding='utf-8') as f:
  json.dump(data, f, ensure_ascii=False, indent=4)

files.download('labeled_data.json')